Building an Ad Optimization Agent is an excellent, practical use case for AI agents\! Since the core functionality will involve complex, multi-step decision-making, the **LangGraph** framework (an extension of LangChain) is ideal for defining the stateful, cyclical workflow.

Here is a simplified Python workshop code template using **LangGraph** and **LangChain** components. This example focuses on the agent's decision-making flow rather than connecting to live ad platform APIs, which would be proprietary and outside a workshop's scope.

You'll need to install the necessary libraries and set your API key (e.g., for an OpenAI or Anthropic LLM).

```bash
pip install langgraph langchain langchain_openai pydantic
```

## Python Workshop Code: Ad Optimization Agent

The agent will follow a core decision loop: **Gather Data** $\rightarrow$ **Analyze** $\rightarrow$ **Decide Action** $\rightarrow$ **Execute Tool** $\rightarrow$ **Repeat/Finish**.

### 1\. Imports and State Definition

We define the `AgentState` to hold the data and decision history throughout the graph's execution.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip -q install langgraph langchain langchain_openai pydantic

In [ ]:
import os
import operator
from typing import TypedDict, Annotated, List

from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END

from google.colab import userdata

# --- Environment Setup (Replace with your actual key) ---
os.environ["OPENAI_API_KEY"] = userdata.get("OpenAI_API_Edu")

# 1. Define the Agent State
# This represents the information passed between nodes in the graph
class AgentState(TypedDict):
    """
    Represents the state of our ad optimization agent.
    - messages: A list of messages/history.
    - campaign_data: The current (simulated) ad campaign metrics.
    - next_action: The recommended action from the analysis.
    """
    messages: Annotated[List[BaseMessage], operator.add]
    campaign_data: list
    budget_allocation: list
    next_action: str
    tool_calls: list # New field to store tool calls

# 2. Initialize LLM
# We'll use a powerful model for the reasoning engine
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

-----

### 2\. Define Custom Tools

We create "tools" (Python functions) that the LLM can decide to use to interact with the external world (e.g., ad platform APIs). In a workshop, these are simulated.

In [ ]:
# --- Simulated Tools for Ad Optimization ---

@tool
def adjust_budget(date: str, channel: str, new_budget_amount: float) -> float:
    """Adjusts the budget for specified channels to the new amount."""
    # Simulate interacting with an external tool/API to adjust the budget
    print(f"Simulating budget adjustment for channel '{channel}' on date {date} to ${new_budget_amount:.2f}") # This print is now inside the tool
    return new_budget_amount

ad_optimization_tools = [adjust_budget]

-----

### 3\. Define Graph Nodes (Steps)

The nodes are the functions that run at each step of the agent's workflow.

In [ ]:
# ---- Imports (put with your other imports) ----
from typing import List, Dict, Any
from datetime import timedelta
import pandas as pd
from langchain_core.messages import HumanMessage, ToolMessage

# ---- Tunable constants (top of cell) ----
SHIFT_PCT = 0.15       # 10–20% suggested; using 15% here
MAX_CHANGE_PCT = 0.20  # guardrail cap per-day change
MIN_FLOOR_PCT = 0.05   # never drop below 5%

# ---- Small helpers ----
def _to_df(records: List[Dict[str, Any]]) -> pd.DataFrame:
    return pd.DataFrame(records or [])

def _to_records(df: pd.DataFrame) -> List[Dict[str, Any]]:
    return df.to_dict(orient="records")

def _end_of_data_guard(messages, campaign_df, budget_df, next_date):
    """Return a FINISH delta if next_date exceeds data; else None."""
    if campaign_df.empty:
        messages.append(HumanMessage(content="No campaign data loaded."))
        return {
            "messages": messages,
            "campaign_data": [],
            "budget_allocation": _to_records(budget_df),
            "tool_calls": [],
            "next_action": "FINISH",
        }
    max_data_date = pd.to_datetime(campaign_df["Date"]).max()
    if next_date > max_data_date:
        messages.append(HumanMessage(content=f"Reached end of data ({max_data_date.date()})."))
        return {
            "messages": messages,
            "campaign_data": _to_records(campaign_df),
            "budget_allocation": _to_records(budget_df),
            "tool_calls": [],
            "next_action": "FINISH",
        }
    return None

# =========================
#          NODES
# =========================

def fetch_campaign_data(state: AgentState) -> dict:
    """Load data once and seed day-1 equal allocation. Subsequent calls are no-ops."""
    print("--- FETCHING DATA ---")

    # Idempotent: if already loaded, skip
    if state.get("campaign_data"):
        print("(fetch_data) campaign_data already present; skipping reload")
        return {}

    try:
        excel_path = "/content/drive/Shareddrives/118S Group Project/Marketing/Mock_Marketing_Data.xlsx"
        campaign_df = pd.read_excel(excel_path)
        campaign_df["Date"] = pd.to_datetime(campaign_df["Date"])
        campaign_df = campaign_df.sort_values("Date")

        # Optional diagnostics for analysis steps later
        campaign_df["CTR"] = campaign_df.apply(
            lambda r: (r["Clicks"] / r["Impressions"]) if r["Impressions"] > 0 else 0, axis=1
        )
        campaign_df["Conversion_Rate"] = campaign_df.apply(
            lambda r: (r["Conversions"] / r["Clicks"]) if r["Clicks"] > 0 else 0, axis=1
        )

        # --- Initialize day-1 equal split (only if missing) ---
        first_date = campaign_df["Date"].min()
        day1_total_spend = float(campaign_df[campaign_df["Date"] == first_date]["Spend"].sum())
        channels = ["Search", "Social", "Display"]
        equal = day1_total_spend / len(channels) if channels else 0.0

        budget_allocation_df = pd.DataFrame({
            "Date": [first_date] * len(channels),
            "Channel": channels,
            "Allocated_Budget": [equal] * len(channels),
        })

        return {
            "campaign_data": _to_records(campaign_df),
            "budget_allocation": _to_records(budget_allocation_df),
        }

    except FileNotFoundError:
        print(f"Error: Excel file not found at {excel_path}")
    except Exception as e:
        print(f"Error reading or processing Excel file: {e}")

    # Fallback (error case)
    return {"campaign_data": [], "budget_allocation": []}


def calculate_daily_allocation(state: AgentState) -> dict:
    """Heuristic allocation with guardrails; prepares tool calls for the next day."""
    print("--- CALCULATING DAILY ALLOCATION ---")
    messages = list(state.get("messages", []))

    budget_df = _to_df(state.get("budget_allocation", []))
    campaign_df = _to_df(state.get("campaign_data", []))

    if "Date" in budget_df.columns:
        budget_df["Date"] = pd.to_datetime(budget_df["Date"])
    if "Date" in campaign_df.columns:
        campaign_df["Date"] = pd.to_datetime(campaign_df["Date"])

    if budget_df.empty:
        msg = "Error: No initial budget allocation found to calculate the next day."
        print(msg)
        messages.append(HumanMessage(content=msg))
        return {
            "messages": messages,
            "campaign_data": _to_records(campaign_df),
            "budget_allocation": _to_records(budget_df),
            "next_action": "FINISH",
        }

    latest_allocated = budget_df["Date"].max()
    next_date = latest_allocated + timedelta(days=1)
    print(f"Next allocation date: {next_date.date()} (based on {latest_allocated.date()})")

    # End-of-data guard
    end_delta = _end_of_data_guard(messages, campaign_df, budget_df, next_date)
    if end_delta:
        return end_delta

    # Use only data available up to latest_allocated (lookback = last full day)
    day_data = campaign_df[campaign_df["Date"] == latest_allocated]
    if day_data.empty:
        # No fresh data → carry forward previous day
        prev = budget_df[budget_df["Date"] == latest_allocated].copy()
        prev["Date"] = next_date
        new_budget_df = pd.concat([budget_df, prev], ignore_index=True)
        reason = f"No campaign data for {latest_allocated.date()} → carry forward to {next_date.date()}."
        messages.append(HumanMessage(content=f"Decision for {next_date.date()}: {reason}"))
        return {
            "messages": messages,
            "campaign_data": _to_records(campaign_df),
            "budget_allocation": _to_records(new_budget_df),
            "tool_calls": [
                {
                    "name": "adjust_budget",
                    "args": {
                        "date": next_date.strftime("%Y-%m-%d"),
                        "channel": row["Channel"],
                        "new_budget_amount": float(row["Allocated_Budget"]),
                    },
                    "id": f"call_adjust_budget_{row['Channel']}_{next_date.strftime('%Y%m%d')}",
                }
                for _, row in prev.iterrows()
            ],
            "next_action": "call_tool",
        }

    # --- Compute performance metric (CVR, fallback CTR) per channel for latest_allocated ---
    perf = []
    for ch, g in day_data.groupby("Channel"):
        clicks = g["Clicks"].sum()
        imps = g["Impressions"].sum()
        conv = g["Conversions"].sum()
        cvr = (conv / clicks) if clicks > 0 else 0.0
        ctr = (clicks / imps) if imps > 0 else 0.0
        metric = cvr if cvr > 0 else ctr
        perf.append((ch, metric))
    channel_perf = dict(perf)

    # If all zeros → equal split
    curr_day_total = float(budget_df.loc[budget_df["Date"] == latest_allocated, "Allocated_Budget"].sum())
    if curr_day_total <= 0:
        prev = budget_df[budget_df["Date"] == latest_allocated].copy()
        prev["Date"] = next_date
        new_budget_df = pd.concat([budget_df, prev], ignore_index=True)
        reason = f"Allocated total on {latest_allocated.date()} was 0 → carry forward to {next_date.date()}."
        messages.append(HumanMessage(content=f"Decision for {next_date.date()}: {reason}"))
        tool_calls = [
            {
                "name": "adjust_budget",
                "args": {
                    "date": next_date.strftime("%Y-%m-%d"),
                    "channel": row["Channel"],
                    "new_budget_amount": float(row["Allocated_Budget"]),
                },
                "id": f"call_adjust_budget_{row['Channel']}_{next_date.strftime('%Y%m%d')}",
            }
            for _, row in prev.iterrows()
        ]
        return {
            "messages": messages,
            "campaign_data": _to_records(campaign_df),
            "budget_allocation": _to_records(new_budget_df),
            "tool_calls": tool_calls,
            "next_action": "call_tool",
        }

    if all(v == 0 for v in channel_perf.values()):
        channels = list(channel_perf.keys())
        equal_split = curr_day_total / len(channels) if channels else 0.0
        next_rows = [{"Date": next_date, "Channel": ch, "Allocated_Budget": equal_split} for ch in channels]
        new_budget_df = pd.concat([budget_df, pd.DataFrame(next_rows)], ignore_index=True)
        reason = f"All channels had zero performance on {latest_allocated.date()} → equal split {next_date.date()}."
    else:
        top = max(channel_perf, key=channel_perf.get)
        # Current %s
        curr = budget_df[budget_df["Date"] == latest_allocated].set_index("Channel")["Allocated_Budget"]
        curr_pct = curr / curr_day_total

        # Shift toward top by SHIFT_PCT, reduce others evenly
        new_pct = curr_pct.copy()
        new_pct[top] = new_pct.get(top, 0.0) + SHIFT_PCT

        others = [c for c in curr_pct.index if c != top]
        if others:
            dec_each = SHIFT_PCT / len(others)
            for c in others:
                new_pct[c] = new_pct.get(c, 0.0) - dec_each

        # Guardrails: per-channel ±MAX_CHANGE_PCT from yesterday
        for c in new_pct.index:
            delta = new_pct[c] - curr_pct.get(c, 0.0)
            if delta > MAX_CHANGE_PCT:
                new_pct[c] = curr_pct.get(c, 0.0) + MAX_CHANGE_PCT
            elif delta < -MAX_CHANGE_PCT:
                new_pct[c] = curr_pct.get(c, 0.0) - MAX_CHANGE_PCT

        # Floors
        for c in new_pct.index:
            if new_pct[c] < MIN_FLOOR_PCT:
                new_pct[c] = MIN_FLOOR_PCT

        # Re-normalize
        s = float(new_pct.sum())
        if s > 0:
            new_pct = new_pct / s
        else:
            # fallback equal
            eq = 1.0 / len(new_pct) if len(new_pct) else 0.0
            new_pct[:] = eq

        # Amounts
        next_amounts = (new_pct * curr_day_total).to_dict()
        next_rows = [{"Date": next_date, "Channel": ch, "Allocated_Budget": amt} for ch, amt in next_amounts.items()]
        new_budget_df = pd.concat([budget_df, pd.DataFrame(next_rows)], ignore_index=True)
        reason = (
            f"Analyzed {latest_allocated.date()} → top: {top}. "
            f"Shifted ~{int(SHIFT_PCT*100)}% toward {top}, "
            f"cap ±{int(MAX_CHANGE_PCT*100)}%, floor {int(MIN_FLOOR_PCT*100)}%."
        )

    # Log + tool calls
    messages.append(HumanMessage(content=f"Decision for {next_date.date()}: {reason}"))
    todays = new_budget_df[new_budget_df["Date"] == next_date]
    tool_calls = [
        {
            "name": "adjust_budget",
            "args": {
                "date": next_date.strftime("%Y-%m-%d"),
                "channel": row["Channel"],
                "new_budget_amount": float(row["Allocated_Budget"]),
            },
            "id": f"call_adjust_budget_{row['Channel']}_{next_date.strftime('%Y%m%d')}",
        }
        for _, row in todays.iterrows()
    ]

    return {
        "messages": messages,
        "campaign_data": _to_records(campaign_df),
        "budget_allocation": _to_records(new_budget_df),
        "tool_calls": tool_calls,
        "next_action": "call_tool",
    }


def execute_tools(state: AgentState) -> dict:
    """Execute prepared tool calls (e.e., adjust_budget)."""
    print("--- EXECUTING TOOL ---")
    messages = list(state.get("messages", []))
    tool_calls = list(state.get("tool_calls", []))
    tool_results: List[ToolMessage] = []

    if not tool_calls:
        messages.append(HumanMessage(content="No budget adjustment tools were called."))
        return {"messages": messages, "next_action": "reasoning"}

    for call in tool_calls:
        name = call.get("name")
        args = call.get("args", {})
        call_id = call.get("id", f"call_{name}")
        tool_obj = next((t for t in ad_optimization_tools if t.name == name), None)
        if not tool_obj:
            tool_results.append(ToolMessage(content=f"Tool '{name}' not found.", tool_call_id=call_id))
            continue
        try:
            result = tool_obj.invoke(args)
            tool_results.append(ToolMessage(content=str(result), tool_call_id=call_id))
        except Exception as e:
            tool_results.append(ToolMessage(content=f"Error executing tool {name}: {e}", tool_call_id=call_id))

    return {"messages": messages + tool_results, "next_action": "reasoning"}


def agent_reasoning(state: AgentState) -> dict:
    """Summarize the latest day's allocation and tool results (no new decisions)."""
    print("--- AGENT REASONING (Final Summary) ---")
    messages = list(state.get("messages", []))
    budget_data = state.get("budget_allocation", [])
    df = _to_df(budget_data)
    if df.empty:
        llm_input = "No budget allocation data was generated or found after tool execution."
    else:
        df["Date"] = pd.to_datetime(df["Date"])
        latest = df["Date"].max()
        day = df[df["Date"] == latest]
        lines = [f"Daily Budget Allocation for {latest.date()}:"]
        for _, row in day.iterrows():
            lines.append(f"- {row['Channel']}: ${float(row['Allocated_Budget']):.2f}")

        # Find the most recent HumanMessage that likely contains the decision reason
        reason_message = next(
            (m for m in reversed(messages)
             if isinstance(m, HumanMessage) and "Decision for" in m.content),
            None
        )
        reason = reason_message.content if reason_message else "No detailed decision reason provided."

        # Find recent ToolMessages
        tool_results = [
            f"- Tool Call ID: {m.tool_call_id}, Result: {m.content}"
            for m in reversed(messages) if isinstance(m, ToolMessage)
        ]
        tool_text = "Tool Execution Results:\n" + ("\n".join(tool_results) if tool_results else "No tool execution results found.")

        # Construct LLM input with only the latest allocation, reason, and tool results
        llm_input = "\n".join(lines) + "\n" + reason + "\n\n" + tool_text + "\n\nPlease summarize concisely."


    system_prompt = (
        "You are an Ad Campaign Budget Allocation Agent. Provide a concise summary of the daily budget allocation, "
        "the rationale, and confirm tool results. Do not make new decisions."
    )
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "{input}")
    ])
    chain = prompt | llm
    response = chain.invoke({"input": llm_input})

    # Append the new AI message to the existing messages list
    return {"messages": messages + [response], "next_action": "FINISH"}



def decide_next_step(state: AgentState) -> str:
    """
    Used only after `agent_reasoning`.
    If the summarizer set FINISH, end; otherwise be defensive.
    """
    na = state.get("next_action")
    if na == "FINISH":
        return "end"
    elif na == "reasoning":
        return "agent_reasoning"
    elif na == "call_tool":
        return "execute_tools"
    # default: end cleanly
    return "end"

# ===================================
#      NEW EVALUATION FUNCTION
# ===================================

def evaluate_performance(state: AgentState) -> dict:
    """
    Analyzes the full simulation run after it's complete.

    1. Calculates baseline (historical) metrics from the campaign data.
    2. Provides a reflection on the agent's heuristic strategy and trade-offs.
    """
    print("\n" + "="*50)
    print("--- 📈 STARTING PERFORMANCE EVALUATION ---")
    print("="*50)

    # --- 1. Load Data from Final State ---
    try:
        # Use the _to_df helper function (assumed to be defined)
        campaign_df = _to_df(state.get("campaign_data", []))
        budget_df = _to_df(state.get("budget_allocation", []))
    except NameError:
        print("Evaluation Error: _to_df helper function not found.")
        return {"evaluation_summary": "Error: _to_df not found."}

    if campaign_df.empty:
        print("Evaluation Failed: No campaign data found in state.")
        return {"evaluation_summary": "No campaign data to evaluate."}

    if budget_df.empty:
        print("Evaluation Failed: No budget allocation data found in state.")
        return {"evaluation_summary": "No budget data to evaluate."}

    # Ensure dates are in datetime format for comparison
    campaign_df["Date"] = pd.to_datetime(campaign_df["Date"])
    budget_df["Date"] = pd.to_datetime(budget_df["Date"])

    # --- 2. Calculate Baseline (Historical) Metrics ---
    # We evaluate the historical data for the *entire period* the agent simulated.
    min_date = budget_df["Date"].min()
    max_date = budget_df["Date"].max()

    # Filter campaign data to match the simulation period
    historical_data = campaign_df[
        (campaign_df["Date"] >= min_date) & (campaign_df["Date"] <= max_date)
    ].copy()

    if historical_data.empty:
        print(f"Evaluation Failed: No historical campaign data found between {min_date.date()} and {max_date.date()}.")
        return {"evaluation_summary": "No matching historical data for simulation period."}

    print(f"Analyzing historical performance from {min_date.date()} to {max_date.date()}...")

    total_spend = historical_data["Spend"].sum()
    total_conversions = historical_data["Conversions"].sum()
    total_clicks = historical_data["Clicks"].sum()

    # Calculate baseline CPAs/CPCs, handling division by zero
    baseline_cpa = total_spend / total_conversions if total_conversions > 0 else 0.0
    baseline_cpc = total_spend / total_clicks if total_clicks > 0 else 0.0

    baseline_report = f"""
    ### 1. Baseline Historical Performance (What Actually Happened)
    This is the "ground truth" performance from your Excel file during the simulation period.

    * **Total Spend:** ${total_spend:,.2f}
    * **Total Conversions:** {total_conversions:,.0f}
    * **Total Clicks:** {total_clicks:,.0f}
    * **Overall Avg. CPA:** ${baseline_cpa:,.2f}
    * **Overall Avg. CPC:** ${baseline_cpc:,.2f}
    """
    print(baseline_report)

    # --- 3. Agent's Strategy & Trade-offs (Reflection) ---

    # Get the agent's *final* decision
    final_allocation = budget_df[budget_df["Date"] == max_date]

    # Get the agent's *first* (equal split) decision
    initial_allocation = budget_df[budget_df["Date"] == min_date]

    # Analyze per-channel historicals to understand *why* the agent shifted
    channel_performance = historical_data.groupby("Channel").agg(
        Total_Spend=("Spend", "sum"),
        Total_Conversions=("Conversions", "sum"),
        Total_Clicks=("Clicks", "sum")
    ).reset_index()

    # Add key metrics the agent (implicitly) uses
    channel_performance["CPA"] = channel_performance.apply(
        lambda r: r["Total_Spend"] / r["Total_Conversions"] if r["Total_Conversions"] > 0 else 0, axis=1
    )
    channel_performance["CVR"] = channel_performance.apply(
        lambda r: r["Total_Conversions"] / r["Total_Clicks"] if r["Total_Clicks"] > 0 else 0, axis=1
    )

    # Format strings for the reflection
    initial_alloc_str = "\n".join(
        [f"    * {row['Channel']}: ${row['Allocated_Budget']:,.2f}" for _, row in initial_allocation.iterrows()]
    )
    final_alloc_str = "\n".join(
        [f"    * {row['Channel']}: ${row['Allocated_Budget']:,.2f}" for _, row in final_allocation.iterrows()]
    )

    # Use pandas.DataFrame.to_markdown for a cleaner table
    try:
        channel_perf_table = channel_performance.to_markdown(index=False, floatfmt=",.2f")
    except ImportError:
        # Fallback if markdown isn't available
        channel_perf_table = channel_performance.to_string(index=False, float_format="{:,.2f}".format)

    channel_perf_table = "        " + channel_perf_table.replace("\n", "\n        ") # Indent for printing

    reflection_report = f"""
    ### 2. Reflection on Agent's Strategy & Trade-offs

    **Agent's Heuristic:** The agent operates on a simple, reactive heuristic:
    1.  Identify the top-performing channel from the *previous day* (based on CVR, falling back to CTR).
    2.  Re-allocate a fixed percentage (`SHIFT_PCT`) of the total budget *from* other channels *to* that top performer.
    3.  Apply guardrails (max change, min floor) to prevent extreme shifts.

    **Observed Activity:**
    * **Initial Allocation (for {min_date.date()}):**
{initial_alloc_str}

    * **Final Allocation (for {max_date.date()}):**
{final_alloc_str}

    **Historical Context (Why the agent shifted):**
    * The agent's decisions were based on *daily* performance. For context, here is the *overall* performance of each channel during the *entire* simulation period:

{channel_perf_table}

    **Trade-offs & Limitations:**
    * **Trade-off:** The agent's strategy is a trade-off between **exploitation** (giving more budget to the proven winner, e.g., 'Search') and **exploration** (by using guardrails, it keeps spending on other channels).
    * **Assumption:** The agent assumes that **past performance (yesterday) is predictive of future performance (today)** and that the top-performing channel is not saturated (i.e., it can absorb more budget efficiently).
    * **CRITICAL LIMITATION:** This evaluation **cannot** determine if the agent's new budget *would have* performed better or worse than the historical baseline. We are analyzing a *reactive agent* on *historical data*, not running a *predictive simulation*. The agent's "Allocated_Budget" is a *decision*; the "Actual_Spend" is a *fact*. Without a model to predict `f(new_budget) -> new_conversions`, we can only evaluate the *logic* of the agent's decisions, not the *outcome*.
    """
    print(reflection_report)
    print("="*50)
    print("--- 🏁 ENDING PERFORMANCE EVALUATION ---")
    print("="*50)

    final_summary = baseline_report + reflection_report
    return {"evaluation_summary": final_summary}

-----

### 4\. Build and Run the Graph

We define the flow using LangGraph's `StateGraph`.

In [ ]:
# Build the LangGraph Workflow
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("fetch_data", fetch_campaign_data)
workflow.add_node("calculate_daily_allocation", calculate_daily_allocation)
workflow.add_node("agent_reasoning", agent_reasoning)
workflow.add_node("execute_tools", execute_tools)

# Set the start point
workflow.set_entry_point("fetch_data")

# Add edges
# After fetching data, we should calculate the first allocation
workflow.add_edge("fetch_data", "calculate_daily_allocation")

# From calculate_daily_allocation, always go to execute_tools
workflow.add_edge("calculate_daily_allocation", "execute_tools")

# After executing a tool, cycle back to reasoning to formulate a summary
workflow.add_edge("execute_tools", "agent_reasoning")


# Conditional edge from reasoning to decide the next step
workflow.add_conditional_edges(
    "agent_reasoning",
    decide_next_step,
    {"call_tool": "execute_tools", "end": END, "calculate_daily_allocation": "calculate_daily_allocation"} # Add transition back to calculate_daily_allocation
)


# Compile the graph
app = workflow.compile()

# ==========================================
# Run the agent across all days
# ==========================================
print("--- START: Ad Optimization (simulate all data days) ---")

state = {
    "messages": [],
    "campaign_data": [],
    "budget_allocation": [],
    "tool_calls": [],
    "next_action": "start", # Initial state for the first invoke
}

# Helpers
def _df(records):
    return pd.DataFrame(records or [])

def _latest_allocated_date(s):
    ba = _df(s.get("budget_allocation"))
    if ba.empty or "Date" not in ba.columns:
        return None
    ba["Date"] = pd.to_datetime(ba["Date"])
    return ba["Date"].max()

def _max_data_date(s):
    df = _df(s.get("campaign_data"))
    if df.empty or "Date" not in df.columns:
        return None
    df["Date"] = pd.to_datetime(df["Date"])
    return df["Date"].max()

# First pass to load data and seed initial allocation.
# The graph will then proceed through allocation -> execute -> reasoning.
state = app.invoke(state, config={"recursion_limit": 50})

max_data = _max_data_date(state)
if max_data is None:
    raise RuntimeError("No campaign data fetched. Check Excel path/columns.")

# ---- ADD THIS LINE ----
# You can manually stop before the dataset ends:
# e.g. end_date_override = "2025-10-15" or None to use full dataset.
end_date_override = "2025-10-5"
if end_date_override:
    max_data = pd.to_datetime(end_date_override)
# ------------------------


simulated = 1
cap = 10000  # safety

print(f"Data range ends at {max_data.date()}")

# Keep invoking one day at a time until we've allocated through the last data day
while simulated < cap:
    latest = _latest_allocated_date(state)
    if latest is None:
         # If budget allocation is somehow lost, refetch and restart the daily process
        print("Warning: Budget allocation missing. Refetching data.")
        state = app.invoke({"messages": state.get("messages", [])}, config={"recursion_limit": 50})
        continue # Restart the loop iteration
    if latest >= max_data:
        print(f"Reached end of data: allocated through {latest.date()}.")
        break

    # For subsequent days, start the graph from the allocation step
    # Pass the current state to continue the process
    state = app.invoke(state, config={"recursion_limit": 50})
    simulated += 1

print(f"--- DONE: Simulated {simulated} day(s) ---")

# ==========================================
# NEW: Call the Evaluation Function
# ==========================================
# This will print the full evaluation report
evaluation_results = evaluate_performance(state)
# ==========================================


# ==========================================
# Final summary print
# ==========================================
msgs = state.get("messages", [])
# Find the last message that is NOT a ToolMessage for the final summary
final_summary = next((m.content for m in reversed(msgs) if not isinstance(m, ToolMessage)), None)


print("\n" + "="*50)
if final_summary:
    print("AGENT FINAL RECOMMENDATION & SUMMARY")
    print(final_summary)
else:
    print("No final summary message available.")
print("="*50)

# (Optional) quick peek at the last few allocations
ba_df = _df(state.get("budget_allocation"))
if not ba_df.empty:
    ba_df["Date"] = pd.to_datetime(ba_df["Date"])
    display(ba_df.sort_values(["Date", "Channel"]).tail(9))

msgs = state.get("messages", [])
# Find the last message that is NOT a ToolMessage for the final summary
final_summary = next((m.content for m in reversed(msgs) if not isinstance(m, ToolMessage)), None)

--- START: Ad Optimization (simulate all data days) ---
--- FETCHING DATA ---
--- CALCULATING DAILY ALLOCATION ---
Next allocation date: 2025-10-04 (based on 2025-10-03)
--- EXECUTING TOOL ---
Simulating budget adjustment for channel 'Search' on date 2025-10-04 to $1776.23
Simulating budget adjustment for channel 'Social' on date 2025-10-04 to $949.36
Simulating budget adjustment for channel 'Display' on date 2025-10-04 to $949.36
--- AGENT REASONING (Final Summary) ---
Data range ends at 2025-10-05
--- FETCHING DATA ---
(fetch_data) campaign_data already present; skipping reload
--- CALCULATING DAILY ALLOCATION ---
Next allocation date: 2025-10-05 (based on 2025-10-04)
--- EXECUTING TOOL ---
Simulating budget adjustment for channel 'Search' on date 2025-10-05 to $2327.47
Simulating budget adjustment for channel 'Social' on date 2025-10-05 to $673.74
Simulating budget adjustment for channel 'Display' on date 2025-10-05 to $673.74
--- AGENT REASONING (Final Summary) ---
Reached end of d

,Date,Channel,Allocated_Budget
2,2025-10-03,Display,1224.986667
0,2025-10-03,Search,1224.986667
1,2025-10-03,Social,1224.986667
5,2025-10-04,Display,949.364667
3,2025-10-04,Search,1776.230667
4,2025-10-04,Social,949.364667
8,2025-10-05,Display,673.742667
6,2025-10-05,Search,2327.474667
7,2025-10-05,Social,673.742667


-----

The video provided below demonstrates building a more advanced research agent using LangChain and LangGraph, offering a great visual example of the framework's capabilities.

[Python Advanced AI Agent Tutorial - LangGraph, LangChain, Firecrawl & More\!](https://www.youtube.com/watch?v=xekw62yQu14&pp=0gcJCf8Ao7VqN5tD) This video is relevant as it provides a practical tutorial on building a complex, multi-step AI agent using the LangGraph and LangChain frameworks, which are central to the workshop code.